In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px


In [8]:
import pandas as pd
import plotly.express as px

# Charger ton fichier (ajuste le chemin si besoin)

df = pd.read_csv("C:\\Users\\Shpet\\Desktop\\data-center-export (2).csv")

df = df.dropna(subset=["Year announced", "Month announced"])
df["Value"] = pd.to_numeric(df["Value"], errors="coerce")
df["Year announced"] = pd.to_numeric(df["Year announced"], errors="coerce")
df["Month announced"] = pd.to_numeric(df["Month announced"], errors="coerce")

# Fusionner année + mois
df["date"] = pd.to_datetime(
    df["Year announced"].astype(int).astype(str) + "-" +
    df["Month announced"].astype(int).astype(str).str.zfill(2),
    format="%Y-%m"
)

# Regrouper les mesures par pays et mois
summary = df.groupby(["Implementer", "date"])["Value"].sum().reset_index()
summary.columns = ["country", "date", "barriers"]

# 🔍 Focus sur la période actuelle
summary = summary[summary["date"].between("2023-01", "2025-12")]
summary["date_str"] = summary["date"].dt.strftime("%Y-%m")

# Palette rouge-orangée
color_scale = [
    [0.0, "#ffffb2"],
    [0.25, "#fecc5c"],
    [0.5, "#fd8d3c"],
    [0.75, "#f03b20"],
    [1.0, "#bd0026"]
]

# Créer la carte animée
fig = px.choropleth(
    summary,
    locations="country",
    locationmode="country names",
    color="barriers",
    animation_frame="date_str",
    color_continuous_scale=color_scale,
    range_color=(0, 150),
    title="🌍 Barrières commerciales annoncées (2023–2025)"
)

# Mise en page + sous-titre + annotation
fig.update_layout(
    geo=dict(
        showframe=False,
        projection_type="natural earth",
        bgcolor='rgba(0,0,0,0)'
    ),
    title_x=0.5,
    width=1000,
    height=600,
    margin=dict(l=0, r=0, t=60, b=50),
    annotations=[
        dict(
            x=0.5,
            y=-0.12,
            xref='paper',
            yref='paper',
            text="Source : Global Trade Alert (avril 2025) • Mise en évidence de l'intensification des tensions géopolitiques",
            showarrow=False,
            font=dict(size=12, color="gray")
        )
    ]
)

# Export en HTML
fig.show()

In [ ]:
fig.write_html("C:\\Users\\Shpet\\Desktop\\barrier_map_2023_2025.html")


,Value,Year announced,Month announced,Implementer,Implementer ISO,date
0,15,2025,2,Brazil,BRA,2025-02-01
1,33,2025,2,France,FRA,2025-02-01
2,13,2025,3,India,IND,2025-03-01
3,13,2025,2,China,CHN,2025-02-01
4,10,2025,1,Japan,JPN,2025-01-01
...,...,...,...,...,...,...
1570,3,2008,10,Germany,DEU,2008-10-01
1571,12,2008,11,China,CHN,2008-11-01
1572,1,2008,10,China,CHN,2008-10-01
1573,26,2008,12,France,FRA,2008-12-01


In [47]:
df_products = pd.read_csv("C:\\Users\\Shpet\\Desktop\\data-center-export (3).csv")


# Nettoyage
df_products = df_products.dropna(subset=["Year announced", "Month announced"])
df_products["Value"] = pd.to_numeric(df_products["Value"], errors="coerce")
df_products["Year announced"] = pd.to_numeric(df_products["Year announced"], errors="coerce")
df_products["Month announced"] = pd.to_numeric(df_products["Month announced"], errors="coerce")

# Créer une colonne date complète (année-mois)
df_products["date"] = pd.to_datetime(
    df_products["Year announced"].astype(int).astype(str) + "-" +
    df_products["Month announced"].astype(int).astype(str).str.zfill(2),
    format="%Y-%m"
)

# Regrouper par pays et date, en sommant les produits affectés
df_summary_products = df_products.groupby(["Implementer", "date"])["Value"].sum().reset_index()
df_summary_products.columns = ["country", "date", "products_affected"]
df_summary_products["date_str"] = df_summary_products["date"].dt.strftime("%Y-%m")

# Limiter à 2023–2025 pour garder la même période de focus
df_summary_products = df_summary_products[
    df_summary_products["date"].between("2023-01", "2025-12")
]

# Créer la carte animée
fig = px.choropleth(
    df_summary_products,
    locations="country",
    locationmode="country names",
    color="products_affected",
    animation_frame="date_str",
    color_continuous_scale="Reds",
    range_color=(0, df_summary_products["products_affected"].max()),  # échelle raisonnable
    title="🌍 Produits affectés par des barrières commerciales annoncées (2023–2025)"
)

fig.update_layout(
    geo=dict(
        showframe=False,
        projection_type="natural earth",
        bgcolor="rgba(0,0,0,0)"
    ),
    title_x=0.5,
    width=1000,
    height=600,
    margin=dict(l=0, r=0, t=60, b=50),
    annotations=[
        dict(
            x=0.5,
            y=-0.12,
            xref='paper',
            yref='paper',
            text="Source : Global Trade Alert • Nombre de produits visés par des mesures commerciales (non pondéré)",
            showarrow=False,
            font=dict(size=12, color="gray")
        )
    ]
)

fig.show()

In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# On relit les deux jeux de données (barrières simples et produits affectés)
df_barriers = pd.read_csv("C:\\Users\\Shpet\\Desktop\\data-center-export interventions.csv")
df_products = pd.read_csv("C:\\Users\\Shpet\\Desktop\\data-center-export produits.csv")

# Fonction de préparation avec vérifications des dates
def prepare_dataframe(df, value_col):
    # Nettoyage
    df = df.dropna(subset=["Year announced", "Month announced"])
    df["Value"] = pd.to_numeric(df["Value"], errors="coerce")
    df["Year announced"] = pd.to_numeric(df["Year announced"], errors="coerce")
    df["Month announced"] = pd.to_numeric(df["Month announced"], errors="coerce")
    
    # Supprimer les mois/années invalides
    df = df[(df["Year announced"] >= 2000) & (df["Year announced"] <= 2100)]
    df = df[(df["Month announced"] >= 1) & (df["Month announced"] <= 12)]

    # Création de la colonne date
    df["date"] = pd.to_datetime(
        df["Year announced"].astype(int).astype(str) + "-" +
        df["Month announced"].astype(int).astype(str).str.zfill(2),
        format="%Y-%m"
    )

    # Agrégation par pays et date
    df_summary = df.groupby(["Implementer", "date"])["Value"].sum().reset_index()
    df_summary.columns = ["country", "date", value_col]
    
    # Garder période utile
    df_summary = df_summary[df_summary["date"].between("2023-01", "2025-12")]

    # Index temporel pour éviter le bug de tri
    df_summary = df_summary.sort_values("date")
    df_summary["month_index"] = df_summary["date"].rank(method="dense").astype(int)
    df_summary["label"] = df_summary["date"].dt.strftime("%Y-%m")

    return df_summary

# Préparation des deux datasets
barriers_df = prepare_dataframe(df_barriers, "barriers")
products_df = prepare_dataframe(df_products, "products_affected")

# Palette de couleurs
color_scale = [
    [0.0, "#ffffb2"],
    [0.25, "#fecc5c"],
    [0.5, "#fd8d3c"],
    [0.75, "#f03b20"],
    [1.0, "#bd0026"]
]

# Mise à jour des figures
fig1 = px.choropleth(
    barriers_df,
    locations="country",
    locationmode="country names",
    color="barriers",
    animation_frame="label",  # on affiche la vraie date
    color_continuous_scale=color_scale,
    range_color=(0, 500),
    title="🌍 Nombre de mesures commerciales annoncées (2023–2025)"
)

fig2 = px.choropleth(
    products_df,
    locations="country",
    locationmode="country names",
    color="products_affected",
    animation_frame="label",
    color_continuous_scale=color_scale,
    range_color=(0, products_df["products_affected"].quantile(0.95)),
    title="🌍 Nombre de produits affectés par des barrières commerciales (2023–2025)"
)

# Export combiné avec labels visibles
with open("C:\\Users\\Shpet\\Desktop\\trade_barriers_with_real_dates.html", "w") as f:
    f.write("<h2>Nombre de mesures commerciales</h2>")
    f.write(fig1.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write("<h2>Nombre de produits affectés</h2>")
    f.write(fig2.to_html(full_html=False, include_plotlyjs=False))


In [10]:
import os
import pandas as pd
import plotly.express as px
from plotly.io import to_image

# Crée un dossier pour les frames
os.makedirs("frames", exist_ok=True)

# On suppose que tu as déjà un DataFrame nommé `barriers_df` avec une colonne "label"
images_paths = []

for label in barriers_df["label"].unique():
    frame_df = barriers_df[barriers_df["label"] == label]

    fig = px.choropleth(
        frame_df,
        locations="country",
        locationmode="country names",
        color="barriers",
        color_continuous_scale="Reds",
        range_color=(0, 500),
        title=f"🌍 Barrières commerciales – {label}"
    )

    img_path = f"frames/frame_{label}.png"
    img_bytes = to_image(fig, format="png", scale=2)
    with open(img_path, "wb") as f:
        f.write(img_bytes)
    
    images_paths.append(img_path)


In [11]:
from moviepy.editor import ImageSequenceClip

clip = ImageSequenceClip(images_paths, fps=1)
clip.write_gif("trade_barriers.gif", fps=1)


ModuleNotFoundError: No module named 'moviepy.editor'